In [1]:
import torch
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, AdamW
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}


In [3]:
alltime = time.time()

# Define the Transformer-based model
class IoTClassifier(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super(IoTClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 768)  # Map input to 768-dim
        self.transformer = BertModel.from_pretrained('bert-base-uncased')
        self.fc2 = torch.nn.Linear(768, num_classes)  # Final output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  
        x = self.transformer(inputs_embeds=x.unsqueeze(1)).last_hidden_state
        x = torch.mean(x, dim=1)  # Pooling
        x = self.fc2(x)  
        return x

# Define custom Dataset class
class IoTDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Feature selection and data loading
feature = [  
    'dstport', 'dstport_class', 'http.chat', 'http.notification', 'http.request.method',
    'ip.flags.df', 'ip.len', 'ip.proto', 'ip.ttl', 'srcport', 'srcport_class', 'tcp.ack',
    'tcp.analysis.ack_rtt', 'tcp.analysis.bytes_in_flight', 'tcp.analysis.initial_rtt',
    'tcp.analysis.push_bytes_sent', 'tcp.completeness', 'tcp.dstport', 'tcp.flags',
    'tcp.flags.push', 'tcp.flags.str', 'tcp.flags.syn', 'tcp.hdr_len', 'tcp.len',
    'tcp.nxtseq', 'tcp.srcport', 'tcp.stream', 'tcp.time_delta', 'tcp.time_relative',
    'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.window_size_value',
    'tls.record.length', 'udp.checksum.status', 'udp.dstport', 'udp.srcport',
    'udp.time_delta', 'udp.time_relative', "Label"
]

for f in file_list:
    print(f"Train: {file_list[f][0]} Test: {file_list[f][1]}")
    train = pd.read_csv(file_list[f][0], usecols=feature)
    test = pd.read_csv(file_list[f][1], usecols=feature)
    
    # Handle missing or invalid values
    train = train.replace(-9999, 0).fillna(0)
    test = test.replace(-9999, 0).fillna(0)
    
    # Separate features and labels
    X_train = train.drop(['Label'], axis=1)
    y_train = train['Label']
    
    # Encode labels and standardize features
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    # Separate features and labels
    X_test = test.drop(['Label'], axis=1)
    y_test = test['Label']
    
    # Encode labels and standardize features
    y_test = label_encoder.fit_transform(y_test)
    X_test = scaler.transform(X_test)
    
    # Create DataLoaders
    batch_size = 16  # Updated batch size
    train_dataset = IoTDataset(X_train, y_train)
    test_dataset = IoTDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # Initialize model, loss function, and optimizer
    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = IoTClassifier(input_dim, num_classes).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0)  # Updated learning rate and weight decay
    
    # Training loop
    num_epochs = 7  # Updated number of epochs
    
    second = time.time()
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')
        
    train_time = float((time.time() - second))
    print(f"train_time: {train_time}")
    second = time.time()
    
    # Evaluation loop with F1-score calculation
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_time = float((time.time() - second))
    print(f"test_time: {test_time}")
    
    # Calculate F1-score
    f1 = f1_score(all_labels, all_preds, average='macro')
    print(f'macro F1-Score: {f1:.4f}')
    
    # Optional: Calculate and print test accuracy
    correct = sum(p == l for p, l in zip(all_preds, all_labels))
    accuracy = 100 * correct / len(all_labels)
    print(f'Test Accuracy: {accuracy:.2f}%')

print(time.time() - alltime)


Train: ./small/AD-S1.csv Test: ./small/DI-S1.csv
Epoch [1/7], Loss: 1.6034
Epoch [2/7], Loss: 0.8258
Epoch [3/7], Loss: 0.6879
Epoch [4/7], Loss: 0.6282
Epoch [5/7], Loss: 0.5902
Epoch [6/7], Loss: 0.5551
Epoch [7/7], Loss: 0.5299
train_time: 13550.62489247322
test_time: 946.7961132526398
macro F1-Score: 0.5366
Test Accuracy: 54.30%
Train: ./small/DI-S1.csv Test: ./small/AD-S1.csv
Epoch [1/7], Loss: 1.7350
Epoch [2/7], Loss: 0.9516
Epoch [3/7], Loss: 0.8347
Epoch [4/7], Loss: 0.7852
Epoch [5/7], Loss: 0.7533
Epoch [6/7], Loss: 0.7191
Epoch [7/7], Loss: 0.6929
train_time: 13841.40567278862
test_time: 255.47177910804749
macro F1-Score: 0.6452
Test Accuracy: 64.25%
Train: ./small/AD-S1.csv Test: ./small/DI-S2.csv
Epoch [1/7], Loss: 1.5836
Epoch [2/7], Loss: 0.8063
Epoch [3/7], Loss: 0.6722
Epoch [4/7], Loss: 0.5989
Epoch [5/7], Loss: 0.5545
Epoch [6/7], Loss: 0.5406
Epoch [7/7], Loss: 0.5147
train_time: 13117.204166412354
test_time: 272.34688663482666
macro F1-Score: 0.5689
Test Accuracy: